In [ ]:
import json
import re

url_pattern = "https?://[\w/:%#\$&\?\(\)~\.=\+\-]+"

with open("./dbp-schema.jsonld", "r") as f:
    jsonld_graph = json.load(f)

contexts = jsonld_graph['@context']
contexts['rdf'] = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'
print(contexts)

def chincha(sbj, obj):
    out = []
    if type(obj) == list:
        for oobj in obj:
            out.extend(chincha(sbj, oobj))
    else:
        if type(obj) == dict:
            obj = obj['@id']
        for k, v in contexts.items():
            obj = obj.replace(k+':', v)
        if re.match(url_pattern, obj):
            out.append("<" + sbj + "> <" + vrb + "> <" + obj + "> .")
        else:  
            out.append("<" + sbj + "> <" + vrb + "> \"" + obj + "\" .")
    return out

result = []

for itm in jsonld_graph['@graph']:
    sbj = itm['@id']
    for k, v in contexts.items():
        sbj = sbj.replace(k+':', v)   
    print(sbj, itm)
    for k, v in itm.items():
        vrb = k
        obj = v
        if k == '@id':
            continue
        if k == '@type':
            vrb = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'
        else:
            for k, v in contexts.items():
                vrb = vrb.replace(k+':', v)
        result.extend(chincha(sbj, obj))

with open("./dbp-schema.nt", "w") as f:
    for line in result:
        f.write(line+"\n")
